In [ ]:
import warnings, os, random, pickle
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
# from tqdm import tqdm
import numpy as np
import pandas as pd
from datetime import datetime
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.models import Sequential, load_model
from keras import Input
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense
from tcn import TCN
from util import timeseries_generator

# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC']='1'
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
import tensorflow as tf
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [ ]:
file_name = "./GSE/GSE6186_cleaned"
file = f"{file_name}.csv"
df = pd.read_csv(file, index_col=0, parse_dates=True)
# df = pd.read_csv(file, index_col=0, parse_dates=True).iloc[:, 0:10]

In [ ]:
kmeans = pickle.load(open("3means.pkl", "rb"))
data = kmeans.cluster_centers_.T
data = pd.DataFrame(data=data, index=df.index)

In [ ]:
test_size = 1
time_steps = 1
b_size = 1

In [ ]:
# _______________________________________________
# ### Defining some constants
# NB_FILTERS = n_features  # Integer. The number of filters to use in the convolutional layers.
NB_FILTERS = 96  # Integer. The number of filters to use in the convolutional layers.
KERNEL_SIZE = 3  # Integer. The size of the kernel to use in each convolutional layer.
NB_STACKS = 1  # Integer. The number of stacks of residual blocks to use
# DILATATIONS = [1, 2, 4, 8]  # List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
DILATATIONS = [1, 2, 4, 8, 16, 32, 64]  # List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
PADDING = 'causal'  # String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and 'same' for a non-causal network.
USE_SKIP_CONNECTION = False  # Boolean. If we want to add skip connections from input to each residual block.
DROPOUT_RATE = 0.05  # Float between 0 and 1. Fraction of the input units to drop.
RETURN_SEQUENCES = False  # Boolean. Whether to return the last output in the output sequence, or the full sequence.
ACTIVATION = 'relu'  # The activation used in the residual blocks o = activation(x + F(x)).
KERNEL_INITIALIZER = 'he_normal'  # Initializer for the kernel weights matrix (Conv1D).
USE_BATCH_NORM = False  # Whether to use batch normalization in the residual layers or not.

def compile_fit_predict(gene, k):
    now = datetime.now().strftime("%Y%m%d-%H%M%S")
    # logdir = f'logs/{now}'
    logdir = f'logs'
    os.makedirs(logdir, exist_ok=True)
    gene = gene.reshape(-1, 1)
    gene = gene.astype('float32')
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    gene = scaler.fit_transform(gene)
    # print(gene)

    X, y = timeseries_generator(pd.DataFrame(gene), time_steps, b_size)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)
    
    """ # print(X_train)
    # print(y_train)
    # print(X_test)
    # print(y_test)
    # print(X_train.shape, y_train.shape)
    # print(X_test.shape, y_test.shape) """
    n_features = 1
    n_input = time_steps
    model = Sequential([
        Input(batch_shape=(None, time_steps, n_features)),
        # Input(batch_shape=(None, X_train.shape[2], X_train.shape[1])),
        TCN(nb_filters=NB_FILTERS,
            kernel_size=KERNEL_SIZE,
            nb_stacks=NB_STACKS,
            dilations=DILATATIONS,
            padding=PADDING,
            use_skip_connections=USE_SKIP_CONNECTION,
            dropout_rate=DROPOUT_RATE,
            return_sequences=RETURN_SEQUENCES,
            activation=ACTIVATION,
            kernel_initializer=KERNEL_INITIALIZER,
            use_batch_norm=USE_BATCH_NORM),  # The TCN layers are here.
        Dense(n_features)
    ])
    # rmse: 0.356
    model.compile(optimizer='adam', loss='mse')
    
    callbacks = [
        # TensorBoard(log_dir=logdir,update_freq=1,histogram_freq=1,write_graph=True,write_images=True),
        ModelCheckpoint(f'{logdir}/best_model.h5', monitor='loss', save_best_only=True, mode='min')
    ]
    model.fit(X_train, y_train, batch_size=100, epochs=100, validation_data=(X_test, y_test), callbacks=callbacks, verbose=0, shuffle=False)
    model = load_model(filepath=f'{logdir}/best_model.h5', custom_objects={'TCN': TCN})

    # KMEANS
    cluster_pred = kmeans.predict(df.T)
    _df = df.iloc[:, np.where(cluster_pred == k)[0]]
    # x = df.values
    RMSE = []
    for col in _df.columns:
        _df[col] = _df[col].astype('float32')
        _X, _y = timeseries_generator(_df[col], time_steps, b_size)
        _X_train, _X_test, _y_train, _y_test = train_test_split(_X, _y, test_size=test_size, shuffle=False)

        y_predicted = model(_X_test)
        # print(y_test.shape, y_predicted.shape)
        y_predicted = scaler.inverse_transform(y_predicted)
        _y_test = _y_test.reshape(-1, 1)
        rmse = mean_squared_error(_y_test, y_predicted, squared=False)
        RMSE.append(rmse)
    rmse = np.mean(RMSE)

    del model
    return rmse

In [ ]:
RMSEs = []
# for col in tqdm(df.columns):
for k, col in enumerate(data.columns):
    gene = np.array(data[col])
    rmse = compile_fit_predict(gene, k)
    print(f'rmse for gene {col}: rmse')
    RMSEs.append(rmse)
    print('so far rmse: %.3f' % np.average(RMSEs))
print('rmse: %.3f' % np.average(RMSEs))